## Inkar-Abruf
Für alle Kreise in Nordbayern:
- Einwohner 65 Jahre und älter (1995 - 2023)
- Pflegebedürftige (2009 - 2023)
- (Durchschnittsalter, Altersgruppen, ...; für Präsentation und Code irrelevant)

Zweiter Abruf über alle Gemeinden in Nordbayern (da Raumbezug = GEM):
- Ärzte (2015 - 2023)

## Zielgruppe
- Kurzfristig breite Bevölkerung, langfristig Kommunal- und Regionalpolitiker
- Bevölkerung hat persönliches Interesse (entweder selbst betroffen oder Angehörige, die betroffen sind oder es bald sein könnten)
- Thema ist bekannt, aber das Publikum kennt die Dringlichkeit noch nicht
- Ziel: Breite Bevölkerung aktivieren und so Druck auf die Politik ausüben
- Dafür: Abholen durch konkrete Beispiele und Zahlen, die den Ernst der Lage verdeutlichen
- Komplexität: niedrig bis mittel; soll für jeden verständlich sein und in einen kurzen Zeitungsartikel passen

## Code

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.figure as figure
import matplotlib.backends.backend_agg as backend
import matplotlib.pyplot as plt
from plotnine import *

Einlesen und Transformieren der Datensätze (bei den Arztpraxen mit Hilfe von KI zur Transformation von Gemeinden zu den entsprechenden Landkreisen)

In [ ]:
# Daten einlesen und Jahreszeile nutzen
altersstruktur_raw = pd.read_excel("C:\\Users\\49159\\Desktop\\Uni\\SDC\\WS2526\\Datenvisualisierung\\Altersstruktur_Nordbayern.xlsx")

jahr_row = altersstruktur_raw.iloc[0]              # Zeile mit Jahren
altersstruktur = altersstruktur_raw.iloc[1:].reset_index(drop=True)

id_vars = ["Kennziffer", "Raumeinheit", "Aggregat"]

In [ ]:
# Arztpraxen aus den Gemeinden einlesen und in langes Format bringen (mit KI)
# Gemeindedatei
a_raw = pd.read_excel("C:\\Users\\49159\\Desktop\\Uni\\SDC\\WS2526\\Datenvisualisierung\\Ärzte Nordbayern.xlsx")
jahr_row = a_raw.iloc[0]
a = a_raw.iloc[1:].reset_index(drop=True)

# 1) Kreis_ID aus Gemeindekennziffer bilden
a["Kreis_ID"] = a["Kennziffer"].astype(str).str.slice(0, 4)

# Altersstruktur-DataFrame
kreise = altersstruktur[["Kennziffer", "Raumeinheit"]].copy()
kreise["Kreis_ID"] = kreise["Kennziffer"].astype(str).str.slice(0, 4)
kreise = kreise.drop_duplicates("Kreis_ID")

# Gemeinden mit Kreisnamen verknüpfen
a_mit_kreis = a.merge(
    kreise[["Kreis_ID", "Raumeinheit"]],
    on="Kreis_ID",
    how="left",
    suffixes=("", "_Kreis")
)

In [ ]:
# erste Zeile: Jahre 2015–2023
jahr_row_a = a_raw.iloc[0]
a = a_raw.iloc[1:].reset_index(drop=True)

id_vars_a = ["Kennziffer", "Raumeinheit", "Aggregat"]

def make_long_a(df, prefix, value_name):
    cols = [c for c in df.columns if str(c).startswith(prefix)]
    tmp = df[id_vars_a + cols].copy()

    long_df = tmp.melt(
        id_vars=id_vars_a,
        value_vars=cols,
        var_name="col",
        value_name=value_name
    )

    long_df["Jahr"] = long_df["col"].map(jahr_row_a).astype(int)
    long_df = long_df.drop(columns="col")
    return long_df

aerzte_long = make_long_a(a, "Ärzte", "aerzte_gem")

In [ ]:
# aerzte_long: Kennziffer (Gemeinde), Raumeinheit (Gemeinde), Jahr, aerzte_gem

# Kreisschlüssel aus Gemeindekennziffer
aerzte_long["Kreis_ID"] = aerzte_long["Kennziffer"].astype(str).str.slice(0, 4)

# Kreisnamen aus Altersstruktur holen
kreise = altersstruktur[["Kennziffer", "Raumeinheit"]].copy()
kreise["Kreis_ID"] = kreise["Kennziffer"].astype(str).str.slice(0, 4)
kreise = kreise.drop_duplicates("Kreis_ID")

# Summe der Ärzte pro Kreis und Jahr
a_kreis = (
    aerzte_long
    .groupby(["Kreis_ID", "Jahr"], as_index=False)["aerzte_gem"]
    .sum()
    .rename(columns={"aerzte_gem": "aerzte"})
    .merge(kreise[["Kreis_ID", "Raumeinheit"]], on="Kreis_ID", how="left")
)

In [ ]:
# Hilfsfunktion und Long-Tabellen (Hilfe von KI)
def make_long(df, prefix, value_name):
    cols = [c for c in df.columns if str(c).startswith(prefix)]
    tmp = df[id_vars + cols].copy()

    long_df = tmp.melt(
        id_vars=id_vars,
        value_vars=cols,
        var_name="col",
        value_name=value_name
    )

    # Jahr zuordnen
    long_df["jahr_str"] = long_df["col"].map(jahr_row)

    # Zeilen ohne Jahr (NaN) entfernen
    long_df = long_df[long_df["jahr_str"].notna()]

    long_df["Jahr"] = long_df["jahr_str"].astype(int)
    long_df = long_df.drop(columns=["col", "jahr_str"])
    return long_df

# 1) Bevölkerung gesamt und erwerbsfähige Bevölkerung
bev_long = make_long(altersstruktur, "Bevölkerung gesamt", "bevölkerung")
erw_long = make_long(
    altersstruktur,
    "Erwerbsfähige Bevölkerung (15 bis unter 65 Jahre)",
    "erwerbsfaehig"
)

# 2) Ü65
u65_long = make_long(altersstruktur, "Einwohner 65 Jahre und älter", "ue65")

# 3) Pflegebedürftige
pflege_long = make_long(altersstruktur, "Pflegebedürftige", "pflegebeduerftig")

Zusätzliche Variablen für Plots

In [ ]:
# Regionaler Mittelwert Ü65 je Jahr
u65_mean = (
    u65_long.groupby("Jahr")["ue65"].mean().reset_index(name="ue65_mean")
)

In [ ]:
# Gesamtzahl der Arztpraxen je Jahr
aerzte_sum = (
    aerzte_long.groupby("Jahr")["aerzte_gem"].sum().reset_index(name="aerzte_sum")
)

In [ ]:
# Anteil der Pflegebedürftigen je Jahr
pflege_mean = (
    pflege_long.groupby("Jahr")["pflegebeduerftig"].mean().reset_index(name="pflege_mean")
)

## Plots

In [ ]:
plt.style.use('seaborn-v0_8-bright')

Beispiel Kulmbach

In [ ]:
# Liniendiagramm der Arztpraxen in Kulmbach
kulmbach = a_kreis[(a_kreis['Raumeinheit'] == 'Kulmbach') & (a_kreis['aerzte'] > 0)].sort_values('Jahr')

start_year = kulmbach['Jahr'].iloc[0]
end_year = kulmbach['Jahr'].iloc[-1]

x = [start_year, end_year]
y = [kulmbach['aerzte'].iloc[0], kulmbach['aerzte'].iloc[-1]]

plt.figure(figsize=(6, 4))
plt.plot(x, y, marker="o")
plt.title("Arztpraxen in Kulmbach - Entwicklung")
plt.xlabel("Jahr")
plt.ylabel("Arztpraxen")
plt.ylim(0, kulmbach['aerzte'].max() + 5)
plt.tight_layout()
plt.show()

In [ ]:
# Liniendiagramm der Ü65-Bevölkerung in Kulmbach

kulmbach = u65_long[u65_long['Raumeinheit'] == 'Kulmbach'].sort_values('Jahr')
x = kulmbach['Jahr']
y = kulmbach['ue65']

plt.figure(figsize=(6, 4))
plt.plot(x, y)
plt.title("Ü65-Bevölkerung in Kulmbach")
plt.xlabel("Jahr")
plt.ylabel("Anteil Ü65-Bevölkerung (in %)")
plt.tight_layout()
plt.show()

In [ ]:
# Ü65-Personen pro Arzt in Kulmbach
# 1) Kulmbach-Daten aus beiden Tabellen holen
a_kul = a_kreis[(a_kreis["Raumeinheit"] == "Kulmbach") & (a_kreis["aerzte"] > 0)].copy()
u_kul = u65_long[u65_long["Raumeinheit"] == "Kulmbach"].copy()
b_kul = bev_long[bev_long["Raumeinheit"] == "Kulmbach"].copy()

# 2) Mergen nach Jahr und Raumeinheit
kul = pd.merge(
    a_kul[["Raumeinheit", "Jahr", "aerzte"]],
    u_kul[["Raumeinheit", "Jahr", "ue65"]],
    on=["Raumeinheit", "Jahr"],
    how="inner",
)

# Einwohner dazu
kul = pd.merge(
    kul,
    b_kul[["Raumeinheit", "Jahr", "bevölkerung"]],
    on=["Raumeinheit", "Jahr"],
    how="inner",
).sort_values("Jahr")

kul = kul.sort_values("Jahr")

# 3) Kennzahl: Ü65-Personen pro Arzt
# Anteil Ü65 wird in Dezimalzahl umgerechnet und dann mit der Bevölkerung multipliziert -> Anzahl Ü65
# danach durch Anzahl Ärzte teilen
kul["ue65_pro_arzt"] = (kul["ue65"] * 1/100) * kul["bevölkerung"] / kul["aerzte"]


# 4 Start- und Endjahr
start_year = kul['Jahr'].iloc[0]
end_year = kul['Jahr'].iloc[-1]

x = [start_year, end_year]
y = [kul['ue65_pro_arzt'].iloc[0], kul['ue65_pro_arzt'].iloc[-1]]

# 5) Plotten
plt.figure(figsize=(6, 4))
plt.plot(x, y, marker="o")
plt.title("Ü65-Personen pro Arzt in Kulmbach")
plt.xlabel("Jahr")
plt.ylabel("Ü65-Personen pro Arzt")
plt.grid(False)
plt.tight_layout()
plt.show()


In [ ]:
# Ü65-Bevölkerung in Kulmbach vs. Nordbayern
kulmbach = u65_long[u65_long['Raumeinheit'] == 'Kulmbach'].sort_values('Jahr')

plt.figure(figsize=(6, 4))
plt.plot(kulmbach['Jahr'], kulmbach['ue65'], label="Kulmbach")
plt.plot(u65_mean['Jahr'], u65_mean['ue65_mean'], label="Nordbayern")
plt.title("Ü65-Bevölkerung in Kulmbach vs. Nordbayern")
plt.xlabel("Jahr")
plt.ylabel("Anteil Ü65-Bevölkerung (in %)")
plt.legend()
plt.tight_layout()
plt.show()

Beispiel Nürnberg

In [ ]:
# Liniendiagramm der Ü65-Bevölkerung in Nürnberg

nuernberg = u65_long[u65_long['Raumeinheit'] == 'Nürnberg'].sort_values('Jahr')
x = nuernberg['Jahr']
y = nuernberg['ue65']

plt.figure(figsize=(6, 4))
plt.plot(x, y)
plt.title("Ü65-Bevölkerung in Nürnberg")
plt.xlabel("Jahr")
plt.ylabel("Anteil Ü65-Bevölkerung (in %)")
plt.tight_layout()
plt.show()

In [ ]:
# Ärzte in Nürnberg
nuernberg = a_kreis[(a_kreis['Raumeinheit'] == 'Nürnberg') & (a_kreis['aerzte'] > 0)].sort_values('Jahr')

x = nuernberg['Jahr']
y = nuernberg['aerzte']

plt.figure(figsize=(6, 4))
plt.plot(x, y)
plt.title("Ärzte in Nürnberg")
plt.xlabel("Jahr") 
plt.ylabel("Anzahl Ärzte pro 10.000 Einwohner")
plt.ylim(0, nuernberg['aerzte'].max() + 5)
plt.tight_layout()
plt.show()

Arztpraxen gesamt in Nordbayern

In [ ]:
# Liniendiagramm Ü65-Struktur in Nordbayern
plt.figure(figsize=(6, 4))
plt.plot(u65_mean['Jahr'], u65_mean['ue65_mean'])
plt.title("Ü65-Anteil in Nordbayern")
plt.xlabel("Jahr")
plt.ylabel("Anteil Personen Ü65 (in %)")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Liniendiagramm Pflegebedürftige in Nordbayern
plt.figure(figsize=(6, 4))
plt.plot(pflege_mean['Jahr'], pflege_mean['pflege_mean'])
plt.title("Pflegebedürftige in Nordbayern")
plt.xlabel("Jahr")
plt.ylabel("Anteil Pflegebedürftiger (in %)")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Liniendiagramm Arztpraxen in Nordbayern
start_year = aerzte_sum["Jahr"].iloc[0]
end_year   = aerzte_sum["Jahr"].iloc[-1]

x = [start_year, end_year]
y = [aerzte_sum["aerzte_sum"].iloc[0], aerzte_sum["aerzte_sum"].iloc[-1]]

plt.figure(figsize=(6, 4))
plt.plot(x, y, marker="o")
plt.title("Arztpraxen in Nordbayern")
plt.xlabel("Jahr")
plt.ylabel("Anzahl Arztpraxen gesamt")
plt.grid(False)
plt.tight_layout()
plt.show()

Ab hier: Vergleich nach Einwohnerzahl

In [ ]:
# Gesamtbevölkerung 2023 je Kreis
bev_23 = (
    bev_long[bev_long["Jahr"] == 2023]
    .groupby("Raumeinheit", as_index=False)["bevölkerung"]
    .sum()
    .rename(columns={"bevölkerung": "bev_2023"})
)

In [ ]:
# Halbieren in "wenige" und "viele" Einwohner
q1 = bev_23["bev_2023"].quantile(1/2)

def kategorie(x):
    if x <= q1:
        return "wenige Einwohner (< 103.269)"
    else:
        return "viele Einwohner (> 103.269)"

bev_23["Einwohner_kat"] = bev_23["bev_2023"].apply(kategorie)
print(q1)

In [ ]:
# Arztzahlen mit Kategorien mergen
a_kreis_cat = a_kreis.merge(
    bev_23[["Raumeinheit", "Einwohner_kat"]],
    on="Raumeinheit",
    how="left"
)

# Summe der Ärzte pro Kategorie und Jahr
a_cat_year = (
    a_kreis_cat
    .groupby(["Einwohner_kat", "Jahr"], as_index=False)["aerzte"]
    .sum()
)

In [ ]:
start_year = a_cat_year["Jahr"].min()
end_year   = a_cat_year["Jahr"].max()

# nur Start- und Endjahr behalten
a_cat_se = a_cat_year[a_cat_year["Jahr"].isin([start_year, end_year])]

# 4) Plot nur mit Start- und Endpunkten (verbunden)
plt.figure(figsize=(7,4))
for kat, grp in a_cat_se.groupby("Einwohner_kat"):
    grp = grp.sort_values("Jahr")
    plt.plot(grp["Jahr"], grp["aerzte"], marker="o", label=kat)

plt.title("Arztpraxen nach Einwohnerkategorie der Kreise\n(Start- und Endjahr)")
plt.xlabel("Jahr")
plt.ylabel("Anzahl Arztpraxen gesamt")
plt.legend(title="Kreistyp")
plt.tight_layout()
plt.show()